In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
from scapy.all import *
import pandas as pd
import numpy as np
import binascii
import seaborn as sns
import matplotlib.pyplot as plt
from util.pandas_util import pcap_to_df
sns.set(color_codes=True)
%matplotlib inline
pd.set_option('display.max_rows', None)

# Analyzing NDT Speedtest

**Objective:** In this section we will examine an [NDT speedtest](https://speed.measurementlab.net/#/) and empirically analyze the how it works.

In [3]:
# read pcaps/ndt_speed_test.pcap using rdpcap, into 'pcap'

In [1]:
print(pcap)

### Writing Processed Output to Disk


In [5]:
# Using pcap_to_df, store the dataframe in memory into 'df'

# Write the 'df' dataframe to disk by saving it as a csv

In [6]:
# Read 'df' from disk

In [2]:
print(df.columns.values)

### Understanding the Data

In [3]:
# Investigating hosts in dataset

# select the src and dst columns from df

# concatenate the src and dst ips into a single dataframe and remove duplicates 


We observe traffic on the 192.168.0.0/16 subnet. This is the subnet of the local network. We can label the direction of the traffic accordingly.

[Documentation for Series](https://pandas.pydata.org/docs/reference/api/pandas.Series.html)

In [10]:
# Labelling direction of traffic

In [11]:
# Filter the df for inbound traffic 

In [12]:
# Filter the df for outbound traffic 

### Viewing Flow Stats

In [4]:
# Look at flow stats 

In [14]:
# Combine port number into a single column

In [15]:
# Filter the df for inbound and outbound traffic

In [5]:
# Look at the proto, src, dst, sport, and dport columns of inbound traffic 

In [6]:
# Group the inbound traffic 

In [7]:
# Group inbound traffic, calculate the total length of bytes transferred 

4.15.166.11 - level3  
151.101.65.195 - fastly  
142.250.68.8 - google  


In [8]:
# Do the same for outbound flows 

In [9]:
# Unique packet lengths in data

### Zooming in on Interesting Flows

In [22]:
# Filter inbound traffic for: 
# source IP '4.15.166.11'
# destination IP '192.168.0.203'
# protocol 6 

In [10]:
# Window time by 1 second

In [11]:
# Sort inbound flow data by window 

In [12]:
# Filter the inbound flow data for packets with a length of 1500

# Group based on the window 

Test is around 10 seconds long

In [15]:
# bytes/sec

In [14]:
# Mbits/sec

In [29]:
# Convert bytes to Mbits 

In [13]:
# Create a line plot, with window on the x axis and Mbits on the y axis 